In [1]:
import os
import glob
# 设置环境变量，让模型下载到D盘
os.environ['HF_HOME'] = 'D:/huggingface_cache/models'
os.environ['HF_HUB_CACHE'] = 'D:/huggingface_cache/hub'
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = '1'
os.environ['HF_HUB_DOWNLOAD_TIMEOUT'] = '1000'


import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers.models.clip import CLIPProcessor, CLIPModel
from transformers.models.auto.modeling_auto import AutoModel
from transformers.models.auto.tokenization_auto import AutoTokenizer
from PIL import Image



d:\miniconda\envs\CLIP\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [72]:
#embed text
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")
clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")

# 测试文本编码
texts = ["photo of a cloud"]
text_inputs = clip_processor(text=texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
#得到text的token id
text_outputs = clip.text_model(**text_inputs)

#print("Text outputs keys:", text_outputs.keys())
hidden_states = text_outputs.last_hidden_state
print("Hidden states shape:", hidden_states.shape)
#hidden_states是[batch_size, sequence_length, hidden_size]
text_embeds = clip.text_projection(hidden_states)


Hidden states shape: torch.Size([1, 6, 512])


In [73]:
#embed image
img = Image.open('G:/Research/James/VideoColBERT/project/try_data/text.jpg')
if img.mode != 'RGB':
    img = img.convert('RGB')
inputs = clip_processor(images=img, return_tensors="pt")
vision_outputs = clip.vision_model(**inputs)
hidden_states = vision_outputs.last_hidden_state
print("Hidden states shape:", hidden_states.shape)
#hidden_states是[batch_size, sequence_length, hidden_size]
#此处输出的sequence_length是197，因为将224*224的图像分割成16*16的patch
#一共196个patch，加上一个CLS,一共197个token.
image_embeds = clip.visual_projection(hidden_states)
print("Image embeddings shape:", image_embeds.shape)

Hidden states shape: torch.Size([1, 197, 768])
Image embeddings shape: torch.Size([1, 197, 512])


In [74]:
#Maxsim
text_embeds_1 = text_embeds.unsqueeze(2).squeeze(0)
print(text_embeds_1.shape)

image_embeds_1 = image_embeds
print(image_embeds_1.shape)

sim_matrix = torch.cosine_similarity(text_embeds_1, image_embeds_1,dim=-1)
print(sim_matrix)
print(sim_matrix.shape)
max_sim = torch.max(sim_matrix,dim=-1).values
print(max_sim)
print(max_sim.shape)
score = torch.sum(max_sim)
print(score)


torch.Size([6, 1, 512])
torch.Size([1, 197, 512])
tensor([[-0.0392, -0.0093, -0.0829,  ...,  0.0332, -0.0126, -0.0107],
        [-0.1171, -0.0922, -0.1244,  ..., -0.0263, -0.0800, -0.0715],
        [-0.1700, -0.1062, -0.1224,  ..., -0.0329, -0.0837, -0.0782],
        [-0.1425, -0.0578, -0.0675,  ..., -0.0140, -0.0734, -0.0758],
        [-0.0562, -0.0100, -0.0562,  ...,  0.0002, -0.0374, -0.0414],
        [-0.0824, -0.0680, -0.0885,  ..., -0.0279, -0.0542, -0.0841]],
       grad_fn=<SumBackward1>)
torch.Size([6, 197])
tensor([0.0545, 0.0536, 0.0655, 0.0769, 0.0705, 0.0211],
       grad_fn=<MaxBackward0>)
torch.Size([6])
tensor(0.3421, grad_fn=<SumBackward0>)


In [ ]:
#1.定义一个compute_score函数
#2.遍历每个query和每个doc
#3.对每个query,return top_3 doc

In [ ]:
from transformers import CLIPProcessor, CLIPModel
